In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns

In [28]:
req = requests.get("https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json")






In [29]:
req.json()

{'displayFieldName': 'NEARESTINTSTREETNAME',
 'fieldAliases': {'OBJECTID': 'OBJECTID',
  'CRIMEID': 'CRASHID',
  'CCN': 'CCN',
  'REPORTDATE': 'REPORTDATE',
  'ROUTEID': 'RouteID',
  'MEASURE': 'Measure',
  'OFFSET': 'OFFSET',
  'STREETSEGID': 'STREETSEGID',
  'ROADWAYSEGID': 'ROADWAYSEGID',
  'FROMDATE': 'FROMDATE',
  'TODATE': 'TODATE',
  'MARID': 'MARID',
  'ADDRESS': 'ADDRESS',
  'LATITUDE': 'LATITUDE',
  'LONGITUDE': 'LONGITUDE',
  'XCOORD': 'XCOORD',
  'YCOORD': 'YCOORD',
  'WARD': 'WARD',
  'EVENTID': 'EventID',
  'MAR_ADDRESS': 'MAR_Address',
  'MAR_SCORE': 'MAR_SCORE',
  'MAJORINJURIES_BICYCLIST': 'MAJORINJURIES_BICYCLIST',
  'MINORINJURIES_BICYCLIST': 'MINORINJURIES_BICYCLIST',
  'UNKNOWNINJURIES_BICYCLIST': 'UNKNOWNINJURIES_BICYCLIST',
  'FATAL_BICYCLIST': 'FATAL_BICYCLIST',
  'MAJORINJURIES_DRIVER': 'MAJORINJURIES_DRIVER',
  'MINORINJURIES_DRIVER': 'MINORINJURIES_DRIVER',
  'UNKNOWNINJURIES_DRIVER': 'UNKNOWNINJURIES_DRIVER',
  'FATAL_DRIVER': 'FATAL_DRIVER',
  'MAJORINJURIE

In [22]:
data = req.json()['features']
raw_df = pd.DataFrame(data)
atts = raw_df.attributes.apply(pd.Series)
geoms = raw_df.geometry.apply(pd.Series)

df2 = pd.concat([atts, geoms], axis=1)

KeyError: 'features'

In [20]:
df.head()

,OBJECTID,CRIMEID,CCN,REPORTDATE,ROUTEID,MEASURE,OFFSET,STREETSEGID,ROADWAYSEGID,FROMDATE,...,OFFINTERSECTION,INTAPPROACHDIRECTION,LOCATIONERROR,LASTUPDATEDATE,MPDLATITUDE,MPDLONGITUDE,MPDGEOX,MPDGEOY,x,y
0,96426045,27760683,18130071,1.533664e+12,58018242,77.873,17.476514,-9.0,33351.0,1533528000000,...,27.265682,None,None,NaN,38.897244,-77.041234,396420.83,136578.25,-77.041261,38.896897
1,96426046,27760684,18130749,1.533664e+12,47039972,51.335,14.930177,-9.0,26479.0,1533614400000,...,61.769244,West,None,NaN,38.964882,-77.001867,399824.31,144096.43,-77.002031,38.964653
2,96426047,27760697,18130778,1.533666e+12,13078322,690.111,19.214300,4942.0,11967.0,1533614400000,...,13.390226,None,None,NaN,38.842184,-76.994091,400548.74,130485.89,-76.993728,38.842348
3,96426048,27760713,18130745,1.533667e+12,12061422,589.264,0.745381,11747.0,12129.0,1533614400000,...,1.737360,None,None,NaN,38.932543,-76.991017,NaN,NaN,-76.991019,38.932544
4,96426049,27760722,18130705,1.533667e+12,13009362,5315.292,28.076962,8276.0,8856.0,1533614400000,...,92.602298,None,None,NaN,38.865260,-76.950346,404199.59,133042.62,-76.951746,38.865432
